In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [2]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='7e2a704c4b5954fe403efd4925e045c672af82b5'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

2017-06-19 10:21:21,701 [INFO] [holmes.experiment.staging] Preparing holmes payload
2017-06-19 10:21:21,702 [INFO] [holmes.experiment.staging] Cloning git:hudl/holmes
2017-06-19 10:21:25,178 [INFO] [holmes.experiment.staging] Checking out git:hudl/holmes:7e2a704c4b5954fe403efd4925e045c672af82b5
2017-06-19 10:21:25,575 [INFO] [holmes.experiment.staging] Preparing distribution package
2017-06-19 10:21:28,230 [INFO] [holmes.experiment.staging] Staging distribution package
2017-06-19 10:21:28,231 [INFO] [holmes.experiment.staging] Commit staged
2017-06-19 10:21:28,232 [INFO] [holmes.experiment.staging] Removing Holmes Directory
2017-06-19 10:21:28,292 [INFO] [holmes.experiment.staging] Extracting default config from package


In [3]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
lost_thresholds = [1]
hungarian_thresholds = [10,15]
minimum_detections = [5,10]

In [136]:
configs = []
list_configs=[]
for lt in lost_thresholds:
    for ht in hungarian_thresholds:
        for min_det in minimum_detections:
            for config in list_dicts:
                current_config_dict = config.copy()
                parameter_map = {"tracker_config.tracklet_config.lost_threshold": lt,"tracker_config.associator_config._class_name": 'SpatialAssociatorConfig', "tracker_config.associator_config.threshold": ht}
                flat_dict = flatten_dict(current_config_dict)
                flat_dict.update(parameter_map)
                updated_config_dict = expand_dict(flat_dict)
                updated_config_dict["filter_configs"] = [{"_class_name": 'BbAreaFilterConfig', 'min_detections': min_det, "max_pixel_area": 5000, "min_pixel_area": 100}]
                configs.append(updated_config_dict)
                file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}-{:02d}-{:02d}.json".format(len(list_configs), 
                                                    current_config_dict['video_config']['video_id'], lt, ht, min_det))
                list_configs.append(file_name)
                with open(file_name, 'w') as f:
                    json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

16 /var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpbgd60a67/M00-ChvUtd-000-01-10-05.json


In [5]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-LT-HT-DET-2-3", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-LT-HT-DET-2-3",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

2017-06-19 10:21:54,739 [INFO] [holmes.experiment.base] Verifying AWS staging area: s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-3


In [6]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [7]:
experiment.stage_experiment()

2017-06-19 10:21:58,364 [INFO] [holmes.experiment.base] staging measurement M00-ChvUtd-000-01-10-05
2017-06-19 10:21:58,776 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpbgd60a67/M00-ChvUtd-000-01-10-05.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-3/deps/M00-ChvUtd-000-01-10-05.json
2017-06-19 10:21:59,124 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpbgd60a67/7e2a704c4b5954fe403efd4925e045c672af82b5.tar.gz, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-3/deps/7e2a704c4b5954fe403efd4925e045c672af82b5.tar.gz
2017-06-19 10:22:00,137 [INFO] [holmes.experiment.base] staging measurement M01-PS0304-001-01-10-05
2017-06-19 10:22:00,662 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpbgd60a67/M01-PS0304-001-01-10-05.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-3/deps

{'experiment_id': 'PTA-95-LT-HT-DET-2-3',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-3/deps/M00-ChvUtd-000-01-10-05.json',
   'experiment_id': 'PTA-95-LT-HT-DET-2-3',
   'job_id': 'PTA-95-LT-HT-DET-2-3-M00-ChvUtd-000-01-10-05',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M00-ChvUtd-000-01-10-05',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-3/deps/7e2a704c4b5954fe403efd4925e045c672af82b5.tar.gz'},
  {'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-3/deps/M01-PS0304-001-01-10-05.json',
   'experiment_id': 'PTA-95-LT-HT-DET-2-3',
   'job_id': 'PTA-95-LT-HT-DET-2-3-M01-PS0304-001-01-10-05',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M01-PS0304-001-01-10-05',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-3/deps/7e2a704c4b5954fe403efd4925e045c672af82

In [8]:
experiment.launch_experiment()

2017-06-19 10:22:14,022 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-3-M00-ChvUtd-000-01-10-05
2017-06-19 10:22:14,283 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-3-M01-PS0304-001-01-10-05
2017-06-19 10:22:14,495 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-3-M02-PP0214-000-01-10-05
2017-06-19 10:22:14,684 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-3-M03-ER0429-001-01-10-05
2017-06-19 10:22:14,868 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-3-M04-ChvUtd-000-01-10-10
2017-06-19 10:22:15,067 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-3-M05-PS0304-001-01-10-10
2017-06-19 10:22:15,260 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-3-M06-PP0214-000-01-10-10
2017-06-19 10:22:15,451 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-3-M07-ER0429-001-01-10-10
2017-06-19 10:22:15,666 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-3-M08-ChvUtd-000-01-15-05
2

{'experiment_id': 'PTA-95-LT-HT-DET-2-3',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-3/deps/M00-ChvUtd-000-01-10-05.json',
   'experiment_id': 'PTA-95-LT-HT-DET-2-3',
   'job_id': 'PTA-95-LT-HT-DET-2-3-M00-ChvUtd-000-01-10-05',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
      'content-length': '105',
      'content-type': 'application/json',
      'date': 'Mon, 19 Jun 2017 15:19:58 GMT',
      'via': '1.1 6eadd6c6c5a53c34c6fce458c34cd790.cloudfront.net (CloudFront)',
      'x-amz-cf-id': 'pUWTOUdavBEURNyB6WbsGG31jabaJJRLf3T5UKStlHJMSNSSxIj1mQ==',
      'x-amzn-requestid': 'c1738b77-5502-11e7-8172-8d9d52863e88',
      'x-amzn-trace-id': 'sampled=0;root=1-5947eb9e-f5a05314199942b460a3e943',
      'x-cache': 'Miss from cloudfront'},
     'HTTPStatusCode': 200,
     'RequestId': 'c1738b77-5502-11e7-8172-8d9d52863e88',


In [23]:
from holmes.io import Retrieve
import re

In [36]:
paths=[]
for m in experiment.measurements:
    measurement_path = m.__dict__['_s3_measurement_path']
    video_id = ''.join(re.match(r'.*/M\d{2}-(\w+)(-{1})(\d+)-.*', measurement_path).groups())
    path = m.__dict__['_s3_measurement_path']+'/trajectory_results_{}.json'.format(video_id)
    paths.append(path)

In [190]:
import pandas as pd
import numpy as np

In [209]:
data = pd.DataFrame({'game':[],'mota':[],'fnr':[],'fpr':[],'mr':[],'lt':[],'ht':[],'det':[],'job_id':[]})
for i,j in enumerate([Retrieve(j) for j in paths]):
    with open(j.local_path, 'r') as fp:
        results = json.load(fp)
        game = results['video_id'][0:6]
        mota = results['MOTA']
        fnr = results['False negative rate']
        fpr = results['False positive rate']
        mr = results['Mismatch rate']
        lt = configs[i]["tracker_config"]["tracklet_config"]["lost_threshold"]
        ht = configs[i]['tracker_config']['associator_config']['threshold']
        det = configs[i]['filter_configs'][0]['min_detections']
        job_id = paths[i][29:73].replace('/','-')
        row = pd.DataFrame({'game':[game],'mota':[mota],'fnr':[fnr],'fpr':[fpr],'mr':[mr],'lt':[lt],'ht':[ht],
                            'det':[det],'job_id':[job_id]})
        data = data.append(row)
data[['lt','ht','det']] = data[['lt','ht','det']].applymap(np.int64)
data

,det,fnr,fpr,game,ht,job_id,lt,mota,mr
0,5,0.117648,0.062582,ChvUtd,10,PTA-95-LT-HT-DET-2-3-M00-ChvUtd-000-01-10-05,1,0.788991,0.014075
0,5,0.091453,0.096575,PS0304,10,PTA-95-LT-HT-DET-2-3-M01-PS0304-001-01-10-05,1,0.765545,0.015267
0,5,0.064231,0.011991,PP0214,10,PTA-95-LT-HT-DET-2-3-M02-PP0214-000-01-10-05,1,0.868661,0.012821
0,5,0.187523,0.047598,ER0429,10,PTA-95-LT-HT-DET-2-3-M03-ER0429-001-01-10-05,1,0.672556,0.024356
0,10,0.137533,0.054737,ChvUtd,10,PTA-95-LT-HT-DET-2-3-M04-ChvUtd-000-01-10-10,1,0.786417,0.009778
0,10,0.108998,0.090956,PS0304,10,PTA-95-LT-HT-DET-2-3-M05-PS0304-001-01-10-10,1,0.766169,0.012071
0,10,0.075446,0.010330,PP0214,10,PTA-95-LT-HT-DET-2-3-M06-PP0214-000-01-10-10,1,0.871528,0.010600
0,10,0.212131,0.038897,ER0429,10,PTA-95-LT-HT-DET-2-3-M07-ER0429-001-01-10-10,1,0.679268,0.019010
0,5,0.110827,0.066717,ChvUtd,15,PTA-95-LT-HT-DET-2-3-M08-ChvUtd-000-01-15-05,1,0.794756,0.009100
0,5,0.088305,0.096469,PS0304,15,PTA-95-LT-HT-DET-2-3-M09-PS0304-001-01-15-05,1,0.767768,0.013757


In [214]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [194]:
output_notebook()

Loading BokehJS ...

In [211]:
data_lt_ht_det = data.copy()
data_lt_ht_det['lost'] = data_lt_ht_det['lt']
data_lt_ht_det['hungarian'] = data_lt_ht_det['ht']
data_lt_ht_det['detections'] = data_lt_ht_det['det']
data_lt_ht_det = data_lt_ht_det.loc[:,['mota','ht', 'lt', 'det', 'lost', 'hungarian', 'detections','fpr','fnr','mr']].groupby(['ht','lt','det']).mean()
data_lt_ht_det

mota  lost  hungarian  detections       fpr       fnr        mr
ht lt det                                                                     
10 1  5    0.773938     1         10           5  0.054686  0.115214  0.016630
      10   0.775845     1         10          10  0.048730  0.133527  0.012865
15 1  5    0.775320     1         15           5  0.056441  0.111351  0.014648
      10   0.779903     1         15          10  0.051953  0.122568  0.012292

In [215]:
scat1 = Scatter(data_lt_ht_det, x='hungarian', y='mota', color='detections', marker='lost')
scat2 = Scatter(data_lt_ht_det, x='hungarian', y='mr', color='detections', marker='lost')
scat3 = Scatter(data_lt_ht_det, x='hungarian', y='fpr', color='detections', marker='lost')
scat4 = Scatter(data_lt_ht_det, x='hungarian', y='fnr', color='detections', marker='lost')
show(column(row(scat1, scat2), row(scat3, scat4)))